<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/6/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Adapted from lab session and https://github.com/coolvision/vae_conv/blob/master/vae_conv_model_mnist.py
# Simple Convolutional Autoencoder

import torch
import torch.nn as nn
import torch.utils as utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
%matplotlib inline

In [2]:

batch_size =100
learning_rate = 0.001

# Download Data

cifar_train = dset.CIFAR10("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
cifar_test  = dset.CIFAR10("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

# Set Data Loader(input pipeline)

train_loader = torch.utils.data.DataLoader(dataset=cifar_train,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=cifar_train,batch_size=batch_size,shuffle=True)

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [41]:
# Encoder 
# torch.nn.Conv2d(in_channels, out_channels, kernel_size,
#                 stride=1, padding=0, dilation=1,
#                 groups=1, bias=True)
# batch x 1 x 28 x 28 -> batch x 512

ngf = 64
ndf = 64
nc = 3

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            # input is (nc) x 28 x 28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, 1024, 4, 1, 0, bias=False),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            # nn.Sigmoid()
        )

        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     1024, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     nc, 4, 2, 1, bias=False),
            # nn.BatchNorm2d(ngf),
            # nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            # nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            # nn.Tanh()
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )

        self.fc1 = nn.Linear(1024, 512)
        self.fc21 = nn.Linear(512, z_dim)
        self.fc22 = nn.Linear(512, z_dim)

        self.fc3 = nn.Linear(z_dim, 512)
        self.fc4 = nn.Linear(512, 1024)

        self.lrelu = nn.LeakyReLU()
        self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        conv = self.encoder(x);
        #print("encode conv", conv.size())
        h1 = self.fc1(conv.view(-1, 1024))
        # print("encode h1", h1.size())
        return self.fc21(h1), self.fc22(h1)

    def decode(self, z):
        #print(z.size())
        h3 = self.relu(self.fc3(z))
        deconv_input = self.fc4(h3)
        #print("deconv_input", deconv_input.size())
        deconv_input = deconv_input.view(-1,1024,1,1)
        #print("deconv_input", deconv_input.size())
        return self.decoder(deconv_input)

    def reparametrize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def forward(self, x):
        # print("x", x.size())
        mu, logvar = self.encode(x)
        # print("mu, logvar", mu.size(), logvar.size())
        z = self.reparametrize(mu, logvar)
        # print("z", z.size())
        #print(z.size())
        decoded = self.decode(z)
        # print("decoded", decoded.size())
        return decoded, mu, logvar



In [50]:
vae = VAE(x_dim=3*32*32, h_dim1= 512, h_dim2=256, z_dim=20)
if torch.cuda.is_available():
    vae.cuda()

optimizer = torch.optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [51]:
# train encoder and decoder
# save and load model

def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        image_n = torch.mul(data+0.25, 0.1 * torch.rand(batch_size,3,32,32))
        if torch.cuda.is_available():
            data = data.cuda()
            image_n = image_n.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(image_n)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [52]:
def test():    
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            if torch.cuda.is_available():
                data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, 51):
    train(epoch)
    test()


Train Epoch: 1 [0/50000 (0%)]	Loss: 2724.283125
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2051.633438
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2022.448750
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2024.079219
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1982.463906
====> Epoch: 1 Average loss: 4041252.2341
====> Test set loss: 4856.3229
Train Epoch: 2 [0/50000 (0%)]	Loss: 1975.844531
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1968.099062
Train Epoch: 2 [20000/50000 (40%)]	Loss: 1935.393906
Train Epoch: 2 [30000/50000 (60%)]	Loss: 1956.893125
Train Epoch: 2 [40000/50000 (80%)]	Loss: 1937.819062
====> Epoch: 2 Average loss: 1965.0009
====> Test set loss: 146857.0166
Train Epoch: 3 [0/50000 (0%)]	Loss: 1970.296094
Train Epoch: 3 [10000/50000 (20%)]	Loss: 1947.793125
Train Epoch: 3 [20000/50000 (40%)]	Loss: 1957.479219


In [ ]:
PATH = 'drive/My Drive/colab_data/cuda_lab/6/vae.pth'
torch.save(vae.state_dict(), PATH)


In [ ]:
    
vae = VAE(x_dim=3*32*32, h_dim1= 512, h_dim2=256, z_dim=2)
vae.load_state_dict(torch.load(PATH))
if torch.cuda.is_available():
  vae.cuda()